In [1]:
from gwsnr import GWSNR
import numpy as np

In [2]:
gwsnr = GWSNR(snr_type='interpolation_aligned_spins_jax')


Initializing GWSNR class...

psds not given. Choosing bilby's default psds
Interpolator will be loaded for L1 detector from ./interpolator_pickle/L1/partialSNR_dict_2.pickle
Interpolator will be loaded for H1 detector from ./interpolator_pickle/H1/partialSNR_dict_2.pickle
Interpolator will be loaded for V1 detector from ./interpolator_pickle/V1/partialSNR_dict_2.pickle

Chosen GWSNR initialization parameters:

npool:  4
snr type:  interpolation_aligned_spins_jax
waveform approximant:  IMRPhenomD
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  9.96
max(mtot) (with the given fmin=20.0): 235.0
detectors:  ['L1', 'H1', 'V1']
psds:  [PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt'), PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/

In [4]:
# gerneral case, random parameters
# add random seed for reproducibility
np.random.seed(0)
nsamples = 1000
mtot = np.random.uniform(2*4.98, 2*112.5,nsamples)
mass_ratio = np.random.uniform(0.2,1,size=nsamples)
param_dict = dict(
    # convert to component masses
    mass_1 = mtot / (1 + mass_ratio),
    mass_2 = mtot * mass_ratio / (1 + mass_ratio),
    # Fix luminosity distance
    luminosity_distance = 500*np.ones(nsamples),
    # Randomly sample everything else:
    theta_jn = np.random.uniform(0,2*np.pi, size=nsamples),
    ra = np.random.uniform(0,2*np.pi, size=nsamples), 
    dec = np.random.uniform(-np.pi/2,np.pi/2, size=nsamples), 
    psi = np.random.uniform(0,2*np.pi, size=nsamples),
    phase = np.random.uniform(0,2*np.pi, size=nsamples),
    geocent_time = 1246527224.169434*np.ones(nsamples),
    a_1 = np.random.uniform(-0.8,0.8, size=nsamples),
    a_2 = np.random.uniform(-0.8,0.8, size=nsamples),
)

In [7]:
snr = gwsnr.snr_with_interpolation(gw_param_dict=param_dict)

In [8]:
# new
%timeit snr = gwsnr.snr_with_interpolation(gw_param_dict=param_dict)

1.81 s ± 350 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# old
%timeit snr = gwsnr.snr_with_interpolation(gw_param_dict=param_dict)

2.42 s ± 58.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
gwsnr.snr_partialsacaled_list.shape

(3, 20, 200, 10, 10)

In [32]:
# Check available attributes and methods
snr_partialsacaled = gwsnr.snr_partialsacaled_list[0]
snr_partialsacaled.min(), snr_partialsacaled.max(), snr_partialsacaled.mean()  

(0.0, 3214.884521484375, 1835.2403855581665)

In [33]:
np.sort(snr_partialsacaled[snr_partialsacaled!=0.0])

array([ 629.57250977,  629.86712646,  630.1663208 , ...,
       3214.83544922, 3214.8828125 , 3214.88452148])

In [34]:
snr_partialsacaled = gwsnr.snr_partialsacaled_list[0]-1835.2403855581665#-629.57250977
snr_partialsacaled.min(), snr_partialsacaled.max(), snr_partialsacaled.mean()  

(-1835.2403855581665, 1379.6441359262085, -2.747401595115662e-13)

In [27]:
np.array([snr_partialsacaled.min(), snr_partialsacaled.max(), snr_partialsacaled.mean()], dtype=np.float32)

array([-629.5725, 2585.312 , 1205.6678], dtype=float32)

In [30]:
scale = np.max(snr_partialsacaled)    # or use mean, as you prefer
snrpartialscaled_array_scaled = (snr_partialsacaled / scale).astype(np.float32)
snrpartialscaled_array_scaled.min(), snrpartialscaled_array_scaled.max(), snrpartialscaled_array_scaled.mean()


(-0.24351896, 1.0, 0.46635294)